In [1]:
import pongGym
import random
import numpy as np
import os
env = pongGym.DoublePong()
PAD_HEIGHT = 80
HALF_PAD_HEIGHT = PAD_HEIGHT // 2
knowframe = 2

pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def encode(state):
    pd1y = state[21]
    pd2y = state[23]

    st = [(max(0, state[i*4+0] + state[i*4+2] * knowframe), abs(state[i*4+1] + state[i*4+3] * knowframe), state[i*4+2], state[i*4+3]) for i in range(5)]
    st.sort(key = lambda s: s[0])

    now = 1
    re = 0
#     print(st)
    for i in range(5):
        ty = 0
        if(st[i][1] > pd1y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd1y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
        ty = 0
        if(st[i][1] > pd2y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd2y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
    return re
state = env.reset()

In [3]:

q_table = np.zeros([(3*3)**5, env.action_space.n])
weight_path = "q_learning_{}.npy".format("exp"+str(knowframe))
if os.path.isfile(weight_path):
    with open(weight_path, 'rb') as f:
        q_table = np.load(f)

In [4]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [5]:
import random
import math
from IPython.display import clear_output

# Hyperparameters
Init_alpha = 0.08
gamma = 0.97
Init_epsilon = 0.2

# For plotting metrics
all_epochs = []
all_penalties = []
all_reward = 0

for i in range(1, 100001):
    alpha = math.exp(-0.0005 * i) * Init_alpha
    epsilon = math.exp(-0.0001 * i) * Init_epsilon

    state = env.reset()
    state = encode(state)
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        next_state, reward, done, info = env.step(action) 
        next_state = encode(next_state)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1
        all_reward += reward
        state = next_state
        epochs += 1
    if i % 100 == 0:
#         clear_output(wait=True)
        print(f"Episode: {i}")
        print(f"average reward: {all_reward / 100}")
        all_reward = 0
        
print("Training finished.\n")

Episode: 100
average reward: 94.54
Episode: 200
average reward: 269.12
Episode: 300
average reward: 325.58
Episode: 400
average reward: 366.41
Episode: 500
average reward: 358.32
Episode: 600
average reward: 453.38
Episode: 700
average reward: 452.8
Episode: 800
average reward: 449.77
Episode: 900
average reward: 447.57
Episode: 1000
average reward: 446.84
Episode: 1100
average reward: 460.13
Episode: 1200
average reward: 459.64
Episode: 1300
average reward: 513.53
Episode: 1400
average reward: 482.68
Episode: 1500
average reward: 500.11
Episode: 1600
average reward: 498.53
Episode: 1700
average reward: 482.75
Episode: 1800
average reward: 544.1
Episode: 1900
average reward: 543.38
Episode: 2000
average reward: 519.36
Episode: 2100
average reward: 551.29
Episode: 2200
average reward: 578.67
Episode: 2300
average reward: 569.14
Episode: 2400
average reward: 559.16
Episode: 2500
average reward: 602.26
Episode: 2600
average reward: 608.8
Episode: 2700
average reward: 602.98
Episode: 2800


Episode: 22100
average reward: 796.14
Episode: 22200
average reward: 796.45
Episode: 22300
average reward: 816.77
Episode: 22400
average reward: 789.07
Episode: 22500
average reward: 793.43
Episode: 22600
average reward: 805.18
Episode: 22700
average reward: 806.43
Episode: 22800
average reward: 807.07
Episode: 22900
average reward: 816.75
Episode: 23000
average reward: 795.93
Episode: 23100
average reward: 807.54
Episode: 23200
average reward: 807.02
Episode: 23300
average reward: 805.94
Episode: 23400
average reward: 800.5
Episode: 23500
average reward: 801.13
Episode: 23600
average reward: 803.65
Episode: 23700
average reward: 791.87
Episode: 23800
average reward: 800.19
Episode: 23900
average reward: 793.36
Episode: 24000
average reward: 812.26
Episode: 24100
average reward: 791.44
Episode: 24200
average reward: 799.11
Episode: 24300
average reward: 789.03
Episode: 24400
average reward: 798.33
Episode: 24500
average reward: 806.45
Episode: 24600
average reward: 785.36
Episode: 2470

Episode: 43800
average reward: 792.47
Episode: 43900
average reward: 802.59
Episode: 44000
average reward: 807.41
Episode: 44100
average reward: 802.96
Episode: 44200
average reward: 785.93
Episode: 44300
average reward: 819.98
Episode: 44400
average reward: 801.82
Episode: 44500
average reward: 815.81
Episode: 44600
average reward: 815.2
Episode: 44700
average reward: 815.39
Episode: 44800
average reward: 808.92
Episode: 44900
average reward: 813.02
Episode: 45000
average reward: 799.83
Episode: 45100
average reward: 790.57
Episode: 45200
average reward: 798.28
Episode: 45300
average reward: 808.14
Episode: 45400
average reward: 787.76
Episode: 45500
average reward: 809.98
Episode: 45600
average reward: 816.37
Episode: 45700
average reward: 795.02
Episode: 45800
average reward: 795.63
Episode: 45900
average reward: 787.02
Episode: 46000
average reward: 791.82
Episode: 46100
average reward: 811.05
Episode: 46200
average reward: 790.32
Episode: 46300
average reward: 804.32
Episode: 4640

Episode: 65500
average reward: 800.82
Episode: 65600
average reward: 812.2
Episode: 65700
average reward: 805.28
Episode: 65800
average reward: 806.47
Episode: 65900
average reward: 813.97
Episode: 66000
average reward: 799.1
Episode: 66100
average reward: 781.49
Episode: 66200
average reward: 808.29
Episode: 66300
average reward: 801.42
Episode: 66400
average reward: 801.97
Episode: 66500
average reward: 799.73
Episode: 66600
average reward: 787.32
Episode: 66700
average reward: 806.02
Episode: 66800
average reward: 810.25
Episode: 66900
average reward: 820.77
Episode: 67000
average reward: 800.75
Episode: 67100
average reward: 813.75
Episode: 67200
average reward: 791.76
Episode: 67300
average reward: 786.48
Episode: 67400
average reward: 793.94
Episode: 67500
average reward: 798.35
Episode: 67600
average reward: 811.53
Episode: 67700
average reward: 805.52
Episode: 67800
average reward: 804.27
Episode: 67900
average reward: 814.83
Episode: 68000
average reward: 806.93
Episode: 68100

Episode: 87200
average reward: 815.39
Episode: 87300
average reward: 799.12
Episode: 87400
average reward: 806.1
Episode: 87500
average reward: 821.46
Episode: 87600
average reward: 792.45
Episode: 87700
average reward: 814.34
Episode: 87800
average reward: 791.14
Episode: 87900
average reward: 799.43
Episode: 88000
average reward: 796.22
Episode: 88100
average reward: 797.75
Episode: 88200
average reward: 797.79
Episode: 88300
average reward: 813.65
Episode: 88400
average reward: 795.82
Episode: 88500
average reward: 793.25
Episode: 88600
average reward: 802.87
Episode: 88700
average reward: 804.83
Episode: 88800
average reward: 803.93
Episode: 88900
average reward: 818.99
Episode: 89000
average reward: 803.92
Episode: 89100
average reward: 807.19
Episode: 89200
average reward: 800.58
Episode: 89300
average reward: 800.67
Episode: 89400
average reward: 813.59
Episode: 89500
average reward: 800.46
Episode: 89600
average reward: 809.68
Episode: 89700
average reward: 811.78
Episode: 8980

In [8]:
with open(weight_path, 'wb') as f:
    np.save(f, q_table)

In [10]:
import time
total_epochs, total_penalties = 0, 0
episodes = 100
total_reward = 0
for _ in range(episodes):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        state = encode(state)
        if reward == -10:
            penalties += 1
        
        epochs += 1
        total_reward += reward
#         env.render()
#         time.sleep(1/60)
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per episode: {total_reward / episodes}")

Results after 100 episodes:
Average timesteps per episode: 10801.0
Average penalties per episode: 18.02
Average reward per episode: 788.34


In [ ]:
with open("weight.coe", 'w') as f:
    f.write("memory_initialization_radix=10;\n")
    f.write("memory_initialization_vector=\n")
    for i in range(3**10):
        f.write(str(np.argmax(q_table[i])))
        if(i == 3**10 - 1):
            f.write(";\n")
        else:
            f.write(",\n")